In [40]:
import re, os, sys, unicodedata
import pandas as pd
import numpy as np
import py_stringmatching as sm
import codecs

In [60]:
def _normalize(data):
    """ Normalize unicode data in Python to remove umlauts, accents, spaces, etc."""
    normal = unicodedata.normalize('NFKD', data).encode('ASCII', 'ignore')
    val = normal.decode("utf-8")
    val = val.lower()
    # remove special characters
    val = re.sub('[^A-Za-z]+', '', val)
    return val

In [44]:
path = '/Users/galvanize/infosys/cip/data/nc_data'
filename0 = 'nc0.csv'
filepath0 = os.path.join(path, filename0)
filename1 = 'nc1.csv'
filepath1 = os.path.join(path, filename1)




In [106]:
df0 = pd.read_csv(filepath0, dtype={'recid':np.int64, 'givenname':str, 'surname':str, 'suburb':str, 'postcode':str})



In [107]:
df1 = pd.read_csv(filepath1, dtype={'recid':np.int64, 'givenname':str, 'surname':str, 'suburb':str, 'postcode':str})

In [108]:
df0.info(), df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   recid      1000000 non-null  int64 
 1   givenname  999928 non-null   object
 2   surname    999980 non-null   object
 3   suburb     999953 non-null   object
 4   postcode   999942 non-null   object
dtypes: int64(1), object(4)
memory usage: 38.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   recid      1000000 non-null  int64 
 1   givenname  999942 non-null   object
 2   surname    999983 non-null   object
 3   suburb     999951 non-null   object
 4   postcode   999943 non-null   object
dtypes: int64(1), object(4)
memory usage: 38.1+ MB


(None, None)

In [109]:
df0.columns

Index(['recid', 'givenname', 'surname', 'suburb', 'postcode'], dtype='object')

In [110]:
df0.columns= [f'{col}0' for col in df0.columns]
df1.columns = [f'{col}1' for col in df1.columns]

In [111]:
df = pd.concat([df0,df1], axis=1)
df = df[['recid0', 'recid1','suburb0','suburb1','givenname0','givenname1','surname0','surname1','postcode0','postcode1' ]]

In [112]:
answers = df[['recid0', 'recid1']]
df = df.drop(['recid0', 'recid1'], axis=1)
df

,suburb0,suburb1,givenname0,givenname1,surname0,surname1,postcode0,postcode1
0,waxhaw,raleigh,kadelyn,ashlev,gragnani,batts,28|73,276o6
1,washington,vass,ronel,tiffamy,carter,perez,2788g,28354
2,millers ceeek,charlogte,jason,david,roehrig,murry,286s1,28268
3,greensboro,rocky mount,antrea,shamkea,mure,hutchinson,27410,278o4
4,jacksonville,louisburg,caedar,john,oliver,snyddr,28542,27s49
...,...,...,...,...,...,...,...,...
999995,charlotte,raleigh,virginia,clyde,angle,wilder,28205,27614
999996,eden,wilmington,james,horace,stone,swain,27288,28405
999997,willow springs,angier,rebecca,peggy,allen,honeycutt,27592,27501
999998,fremont,elm city,julie,ethel,cummings,hamm,27830,27822


In [113]:
for col in df.columns:
    df.loc[:,col] = df[col].apply(lambda x: str(x)).str.replace('[^A-Za-z0-9]+', '', regex=True)
df.head()

,suburb0,suburb1,givenname0,givenname1,surname0,surname1,postcode0,postcode1
0,waxhaw,raleigh,kadelyn,ashlev,gragnani,batts,2873,276o6
1,washington,vass,ronel,tiffamy,carter,perez,2788g,28354
2,millersceeek,charlogte,jason,david,roehrig,murry,286s1,28268
3,greensboro,rockymount,antrea,shamkea,mure,hutchinson,27410,278o4
4,jacksonville,louisburg,caedar,john,oliver,snyddr,28542,27s49


In [114]:
tok = sm.QgramTokenizer(qval=3, return_set=True, padding=False)

In [115]:
df['suburb0'].apply(lambda x: set(tok.tokenize(x)))

0                                      {xha, haw, wax, axh}
1                  {shi, gto, ton, ngt, ing, ash, was, hin}
2         {eee, ill, mil, ler, ers, sce, cee, lle, eek, ...
3                  {ens, gre, ree, sbo, nsb, oro, bor, een}
4         {cks, ill, nvi, vil, onv, son, kso, ack, lle, ...
                                ...                        
999995                  {har, ott, rlo, arl, lot, cha, tte}
999996                                           {den, ede}
999997    {pri, ows, ill, wsp, wil, ngs, ing, spr, rin, ...
999998                            {rem, mon, ont, fre, emo}
999999                       {erd, dee, rde, abe, ber, een}
Name: suburb0, Length: 1000000, dtype: object

In [116]:
for col in df.columns:
    df.loc[:,f'{col}_qgrams'] = df[col].apply(lambda x: set(tok.tokenize(x)))
df.head()

,suburb0,suburb1,givenname0,givenname1,surname0,surname1,postcode0,postcode1,suburb0_qgrams,suburb1_qgrams,givenname0_qgrams,givenname1_qgrams,surname0_qgrams,surname1_qgrams,postcode0_qgrams,postcode1_qgrams
0,waxhaw,raleigh,kadelyn,ashlev,gragnani,batts,2873,276o6,"{xha, haw, wax, axh}","{lei, ral, ale, eig, igh}","{ely, ade, del, kad, lyn}","{shl, lev, ash, hle}","{nan, gna, agn, ani, rag, gra}","{bat, tts, att}","{873, 287}","{76o, 6o6, 276}"
1,washington,vass,ronel,tiffamy,carter,perez,2788g,28354,"{shi, gto, ton, ngt, ing, ash, was, hin}","{vas, ass}","{nel, ron, one}","{iff, amy, tif, fam, ffa}","{art, car, rte, ter}","{rez, per, ere}","{88g, 278, 788}","{354, 835, 283}"
2,millersceeek,charlogte,jason,david,roehrig,murry,286s1,28268,"{eee, ill, mil, ler, ers, sce, cee, lle, eek, ...","{har, rlo, arl, gte, ogt, log, cha}","{jas, son, aso}","{avi, dav, vid}","{roe, ehr, rig, hri, oeh}","{mur, urr, rry}","{286, 86s, 6s1}","{282, 268, 826}"
3,greensboro,rockymount,antrea,shamkea,mure,hutchinson,27410,278o4,"{ens, gre, ree, sbo, nsb, oro, bor, een}","{ock, roc, kym, mou, unt, ymo, oun, cky}","{rea, ntr, tre, ant}","{kea, ham, amk, mke, sha}","{mur, ure}","{chi, hut, nso, son, ins, tch, hin, utc}","{274, 741, 410}","{278, 8o4, 78o}"
4,jacksonville,louisburg,caedar,john,oliver,snyddr,28542,27s49,"{cks, ill, nvi, vil, onv, son, kso, ack, lle, ...","{uis, oui, isb, lou, bur, sbu, urg}","{dar, cae, aed, eda}","{ohn, joh}","{oli, ver, liv, ive}","{nyd, ddr, sny, ydd}","{285, 542, 854}","{27s, s49, 7s4}"


In [118]:
from collections import defaultdict, namedtuple

In [ ]:
# QGRAMS = namedtuple('QGRAMS','suburb, givenname, surname, postcode') 
dd = defaultdict(list)
# for value, key in zip(prefix.index, prefix.values):
#     dd[key].append(value)

for value, keys in zip(df.index, df.set_of_trigrams):
    for key in keys:
        dd[key].append(value)
len(dd)

In [119]:
QGRAMS = namedtuple('QGRAMS','suburb, givenname, surname, postcode') 

In [123]:
data0 = QGRAMS(df['suburb0_qgrams'],df['givenname0_qgrams'],df['surname0_qgrams'],df['postcode0_qgrams'])

In [124]:
data1 = QGRAMS(df['suburb1_qgrams'],df['givenname1_qgrams'],df['surname1_qgrams'],df['postcode1_qgrams'])

In [126]:
data0.suburb

0                                      {xha, haw, wax, axh}
1                  {shi, gto, ton, ngt, ing, ash, was, hin}
2         {eee, ill, mil, ler, ers, sce, cee, lle, eek, ...
3                  {ens, gre, ree, sbo, nsb, oro, bor, een}
4         {cks, ill, nvi, vil, onv, son, kso, ack, lle, ...
                                ...                        
999995                  {har, ott, rlo, arl, lot, cha, tte}
999996                                           {den, ede}
999997    {pri, ows, ill, wsp, wil, ngs, ing, spr, rin, ...
999998                            {rem, mon, ont, fre, emo}
999999                       {erd, dee, rde, abe, ber, een}
Name: suburb0_qgrams, Length: 1000000, dtype: object

In [127]:
data1.suburb

0                        {lei, ral, ale, eig, igh}
1                                       {vas, ass}
2              {har, rlo, arl, gte, ogt, log, cha}
3         {ock, roc, kym, mou, unt, ymo, oun, cky}
4              {uis, oui, isb, lou, bur, sbu, urg}
                            ...                   
999995                   {lei, ral, ale, eig, igh}
999996    {min, wil, gto, ton, ngt, ing, lmi, ilm}
999997                        {ngi, gie, ier, ang}
999998                   {mci, lmc, cit, ity, elm}
999999         {ann, ner, rel, ban, nne, ere, elk}
Name: suburb1_qgrams, Length: 1000000, dtype: object